# Import statement

In [85]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ast
from operator import itemgetter

#Natural language toolkit
import nltk
from nltk.stem import PorterStemmer

#libraries to build vectors with dimensions which are every single words
from sklearn.feature_extraction.text import CountVectorizer

#Cosin library to calculate the angle of every two vector
from sklearn.metrics.pairwise import cosine_similarity


# 1. Loading data

In [20]:
path = '../Data'
movies = pd.read_csv(path + '/movies_metadata.csv')
credits = pd.read_csv(path + '/credits.csv')
keywords = pd.read_csv(path + '/keywords.csv')
ratings = pd.read_csv(path + '/ratings_small.csv')

C:\Users\baomi\AppData\Local\Temp\ipykernel_4524\2584738648.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies = pd.read_csv(path + '/movies_metadata.csv')


In [21]:
print(movies.shape)
movies.head(2)

(45466, 24)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


In [22]:
print(credits.shape)
credits.head(2)

(45476, 3)


,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844


In [23]:
print(keywords.shape)
keywords.head(2)

(46419, 2)


,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."


In [24]:
print(ratings.shape)
ratings.head(2)

(100004, 4)


,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179


In [25]:
ratings['rating'].max()

5.0

In [26]:
movies.isnull().sum()

adult                        0
belongs_to_collection    40972
budget                       0
genres                       0
homepage                 37684
id                           0
imdb_id                     17
original_language           11
original_title               0
overview                   954
popularity                   5
poster_path                386
production_companies         3
production_countries         3
release_date                87
revenue                      6
runtime                    263
spoken_languages             6
status                      87
tagline                  25054
title                        6
video                        6
vote_average                 6
vote_count                   6
dtype: int64

In [27]:
poster_path = movies[['id','original_title','poster_path']]
poster_path.head(1)

,id,original_title,poster_path
0,862,Toy Story,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg


In [28]:
movies = movies[['id','original_title','genres','overview','production_companies','production_countries']]

In [29]:
movies.head(1)

,id,original_title,genres,overview,production_companies,production_countries
0,862,Toy Story,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...","Led by Woody, Andy's toys live happily in his ...","[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o..."


In [30]:
credits.id = credits.id.astype(str)
keywords.id = keywords.id.astype(str)
movies = movies.merge(credits,on = 'id').merge(keywords, on = 'id')
movies.head(1)

,id,original_title,genres,overview,production_companies,production_countries,cast,crew,keywords
0,862,Toy Story,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...","Led by Woody, Andy's toys live happily in his ...","[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...","[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."


In [31]:
movies.overview.fillna('',inplace=True)
movies.dropna(inplace= True)

In [32]:
print(movies.shape)
movies.isnull().sum()

(46624, 9)


id                      0
original_title          0
genres                  0
overview                0
production_companies    0
production_countries    0
cast                    0
crew                    0
keywords                0
dtype: int64

In [33]:
movies.cast[0]

"[{'cast_id': 14, 'character': 'Woody (voice)', 'credit_id': '52fe4284c3a36847f8024f95', 'gender': 2, 'id': 31, 'name': 'Tom Hanks', 'order': 0, 'profile_path': '/pQFoyx7rp09CJTAb932F2g8Nlho.jpg'}, {'cast_id': 15, 'character': 'Buzz Lightyear (voice)', 'credit_id': '52fe4284c3a36847f8024f99', 'gender': 2, 'id': 12898, 'name': 'Tim Allen', 'order': 1, 'profile_path': '/uX2xVf6pMmPepxnvFWyBtjexzgY.jpg'}, {'cast_id': 16, 'character': 'Mr. Potato Head (voice)', 'credit_id': '52fe4284c3a36847f8024f9d', 'gender': 2, 'id': 7167, 'name': 'Don Rickles', 'order': 2, 'profile_path': '/h5BcaDMPRVLHLDzbQavec4xfSdt.jpg'}, {'cast_id': 17, 'character': 'Slinky Dog (voice)', 'credit_id': '52fe4284c3a36847f8024fa1', 'gender': 2, 'id': 12899, 'name': 'Jim Varney', 'order': 3, 'profile_path': '/eIo2jVVXYgjDtaHoF19Ll9vtW7h.jpg'}, {'cast_id': 18, 'character': 'Rex (voice)', 'credit_id': '52fe4284c3a36847f8024fa5', 'gender': 2, 'id': 12900, 'name': 'Wallace Shawn', 'order': 4, 'profile_path': '/oGE6JqPP2xH4t

In [34]:
movies.keywords[0]

"[{'id': 931, 'name': 'jealousy'}, {'id': 4290, 'name': 'toy'}, {'id': 5202, 'name': 'boy'}, {'id': 6054, 'name': 'friendship'}, {'id': 9713, 'name': 'friends'}, {'id': 9823, 'name': 'rivalry'}, {'id': 165503, 'name': 'boy next door'}, {'id': 170722, 'name': 'new toy'}, {'id': 187065, 'name': 'toy comes to life'}]"

In [35]:
def convert(text):
    list = []
    for i in ast.literal_eval(text):
        list.append(i['name'])
    return list

In [36]:
movies.genres = movies.genres.apply(convert)
movies.production_companies = movies.production_companies.apply(convert)
movies.production_countries = movies.production_countries.apply(convert)
movies.cast = movies.cast.apply(convert)
movies.keywords = movies.keywords.apply(convert)

In [37]:
movies.head(1)

,id,original_title,genres,overview,production_companies,production_countries,cast,crew,keywords
0,862,Toy Story,"[Animation, Comedy, Family]","Led by Woody, Andy's toys live happily in his ...",[Pixar Animation Studios],[United States of America],"[Tom Hanks, Tim Allen, Don Rickles, Jim Varney...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[jealousy, toy, boy, friendship, friends, riva..."


In [38]:
#unique countries
set(movies.production_countries.sum())

{'Afghanistan',
 'Albania',
 'Algeria',
 'Angola',
 'Antarctica',
 'Argentina',
 'Armenia',
 'Aruba',
 'Australia',
 'Austria',
 'Azerbaijan',
 'Bahamas',
 'Bangladesh',
 'Barbados',
 'Belarus',
 'Belgium',
 'Bermuda',
 'Bhutan',
 'Bolivia',
 'Bosnia and Herzegovina',
 'Botswana',
 'Brazil',
 'Brunei Darussalam',
 'Bulgaria',
 'Burkina Faso',
 'Cambodia',
 'Cameroon',
 'Canada',
 'Cayman Islands',
 'Chad',
 'Chile',
 'China',
 'Colombia',
 'Congo',
 'Costa Rica',
 "Cote D'Ivoire",
 'Croatia',
 'Cuba',
 'Cyprus',
 'Czech Republic',
 'Czechoslovakia',
 'Denmark',
 'Dominican Republic',
 'East Germany',
 'Ecuador',
 'Egypt',
 'El Salvador',
 'Estonia',
 'Ethiopia',
 'Finland',
 'France',
 'French Polynesia',
 'French Southern Territories',
 'Georgia',
 'Germany',
 'Ghana',
 'Gibraltar',
 'Greece',
 'Guatemala',
 'Guinea',
 'Honduras',
 'Hong Kong',
 'Hungary',
 'Iceland',
 'India',
 'Indonesia',
 'Iran',
 'Iraq',
 'Ireland',
 'Israel',
 'Italy',
 'Jamaica',
 'Japan',
 'Jordan',
 'Kazakhst

In [39]:
movies.crew[0]

'[{\'credit_id\': \'52fe4284c3a36847f8024f49\', \'department\': \'Directing\', \'gender\': 2, \'id\': 7879, \'job\': \'Director\', \'name\': \'John Lasseter\', \'profile_path\': \'/7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg\'}, {\'credit_id\': \'52fe4284c3a36847f8024f4f\', \'department\': \'Writing\', \'gender\': 2, \'id\': 12891, \'job\': \'Screenplay\', \'name\': \'Joss Whedon\', \'profile_path\': \'/dTiVsuaTVTeGmvkhcyJvKp2A5kr.jpg\'}, {\'credit_id\': \'52fe4284c3a36847f8024f55\', \'department\': \'Writing\', \'gender\': 2, \'id\': 7, \'job\': \'Screenplay\', \'name\': \'Andrew Stanton\', \'profile_path\': \'/pvQWsu0qc8JFQhMVJkTHuexUAa1.jpg\'}, {\'credit_id\': \'52fe4284c3a36847f8024f5b\', \'department\': \'Writing\', \'gender\': 2, \'id\': 12892, \'job\': \'Screenplay\', \'name\': \'Joel Cohen\', \'profile_path\': \'/dAubAiZcvKFbboWlj7oXOkZnTSu.jpg\'}, {\'credit_id\': \'52fe4284c3a36847f8024f61\', \'department\': \'Writing\', \'gender\': 0, \'id\': 12893, \'job\': \'Screenplay\', \'name\': \'A

In [40]:
def convert_crew(text):
    list = []
    for i in ast.literal_eval(text):
        list.append(itemgetter('job','name')(i))
    mytuple = dict((x, y) for x, y in list)
    return mytuple
convert_crew(movies.crew[0])

{'Director': 'John Lasseter',
 'Screenplay': 'Alec Sokolow',
 'Producer': 'Ralph Guggenheim',
 'Executive Producer': 'Steve Jobs',
 'Editor': 'Robert Gordon',
 'Art Direction': 'Ralph Eggleston',
 'Foley Editor': 'Mary Helen Leasman',
 'Animation': 'Ken Willard',
 'ADR Editor': 'Marilyn McCoppen',
 'Orchestrator': 'Don Davis',
 'Color Timer': 'Dale E. Grahn',
 'CG Painter': 'William Cone',
 'Original Story': 'Andrew Stanton',
 'Post Production Supervisor': 'Patsy Bouge',
 'Sculptor': 'Shelley Daniels Lekven',
 'Animation Director': 'Rich Quade',
 'Music': 'Randy Newman',
 'Layout': 'Desirée Mourad',
 'Music Editor': 'James Flamberg',
 'Negative Cutter': 'Rick Mackay',
 'Title Designer': 'Susan Bradley',
 'Supervising Technical Director': 'William Reeves',
 'Songs': 'Randy Newman',
 'Supervising Animator': 'Pete Docter',
 'Sound Designer': 'Gary Rydstrom',
 'Production Supervisor': 'Karen Robert Jackson',
 'Executive Music Producer': 'Chris Montan',
 'Visual Effects Supervisor': 'Thomas

In [41]:
movies.crew = movies.crew.apply(convert_crew)
movies.crew = movies.crew.apply(lambda x: x.get('Director'))

In [42]:
movies.head(1)

,id,original_title,genres,overview,production_companies,production_countries,cast,crew,keywords
0,862,Toy Story,"[Animation, Comedy, Family]","Led by Woody, Andy's toys live happily in his ...",[Pixar Animation Studios],[United States of America],"[Tom Hanks, Tim Allen, Don Rickles, Jim Varney...",John Lasseter,"[jealousy, toy, boy, friendship, friends, riva..."


In [43]:
def remove_space(words):
    list = []
    for i in words:
        list.append(i.replace(' ',''))
    return list

In [44]:
movies.genres = movies.genres.apply(remove_space)
movies.cast = movies.cast.apply(remove_space)
movies.keywords = movies.keywords.apply(remove_space)
movies.production_companies = movies.production_companies.apply(remove_space)
movies.production_countries = movies.production_countries.apply(remove_space)

In [45]:
movies.genres = movies.genres.apply(' '.join)
movies.production_companies = movies.production_companies.apply(' '.join)
movies.production_countries = movies.production_countries.apply(' '.join)
movies.cast = movies.cast.apply(' '.join)
movies.keywords = movies.keywords.apply(' '.join)

In [46]:
movies.isnull().sum()

id                        0
original_title            0
genres                    0
overview                  0
production_companies      0
production_countries      0
cast                      0
crew                    917
keywords                  0
dtype: int64

In [47]:
movies.crew.fillna('',inplace=True)

In [48]:
movies.isnull().sum()

id                      0
original_title          0
genres                  0
overview                0
production_companies    0
production_countries    0
cast                    0
crew                    0
keywords                0
dtype: int64

In [53]:
movies['tags'] = movies.genres +' '+ movies.overview +' '+ movies.production_companies +' '+ movies.production_countries +' '+ movies.cast +' '+ movies.crew +' '+ movies.keywords

In [54]:
preprocessed_movies = movies[['id','original_title','tags']]

In [56]:
preprocessed_movies.tags[0]

"Animation Comedy Family Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences. PixarAnimationStudios UnitedStatesofAmerica TomHanks TimAllen DonRickles JimVarney WallaceShawn JohnRatzenberger AnniePotts JohnMorris ErikvonDetten LaurieMetcalf R.LeeErmey SarahFreeman PennJillette John Lasseter jealousy toy boy friendship friends rivalry boynextdoor newtoy toycomestolife"

In [57]:
ps = PorterStemmer()

In [58]:
# format the some word to be harmonized such as "goes" -> "go"
def stems(text):
    l = []
    for i in text.split():
        l.append(ps.stem(i))
    return " ".join(l)

In [75]:
preprocessed_movies.tags = preprocessed_movies.tags.apply(stems)

# type(preprocessed_movies.tags)
# count = 0
# for i in preprocessed_movies.tags:
#     count +=1
#     print(count, type(i))
# # 636 662 1129 1294

C:\Users\baomi\AppData\Local\Temp\ipykernel_4524\3342832998.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  preprocessed_movies.tags = preprocessed_movies.tags.apply(stems)


In [76]:
preprocessed_movies.tags[0]

"anim comedi famili led by woody, andy' toy live happili in hi room until andy' birthday bring buzz lightyear onto the scene. afraid of lose hi place in andy' heart, woodi plot against buzz. but when circumst separ buzz and woodi from their owner, the duo eventu learn to put asid their differences. pixaranimationstudio unitedstatesofamerica tomhank timallen donrickl jimvarney wallaceshawn johnratzenberg anniepott johnmorri erikvondetten lauriemetcalf r.leeermey sarahfreeman pennjillett john lasset jealousi toy boy friendship friend rivalri boynextdoor newtoy toycomestolif"

In [74]:
cvec = CountVectorizer(max_features=5000, stop_words = 'english')

In [79]:
vector = cvec.fit_transform(preprocessed_movies.tags).toarray()

In [86]:
vector

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [87]:
correlation_matrix = cosine_similarity(vector)

MemoryError: Unable to allocate 16.2 GiB for an array with shape (46624, 46624) and data type float64

In [ ]:
correlation_matrix
# it seem to be overloaded because of the number of data is too big!!! let solve this